In [ ]:
from dotenv import load_dotenv

load_dotenv(".env")

## Query all your positions


In [ ]:
from agent0 import Chain, Hyperdrive
import os

# PUBLIC_ADDRESS = "0xPUBLIC_ADDRESS"
# RPC_URI = "https://sepolia.rpc.url"
PUBLIC_ADDRESS = os.getenv("PUBLIC_ADDRESS")
RPC_URI = os.getenv("RPC_URI", "")

# Address of Hyperdrive Sepolia registry
REGISTRY_ADDRESS = "0x03f6554299acf544ac646305800f57db544b837a"

## View open and closed positions in all pools
chain = Chain(RPC_URI)
agent = chain.init_agent(public_address=PUBLIC_ADDRESS)
registered_pools = Hyperdrive.get_hyperdrive_pools_from_registry(
    chain,
    registry_address=REGISTRY_ADDRESS,
)
agent.get_positions(pool_filter=registered_pools, show_closed_positions=True)

## Automate withdrawing funds from matured positions


In [ ]:
# PRIVATE_KEY = "0xPRIVATE_KEY"
PRIVATE_KEY = os.getenv("PRIVATE_KEY")

# Initialize agent with private key for transactions
agent = chain.init_agent(private_key=PRIVATE_KEY)
for pool in registered_pools:
    # Close all mature longs
    for long in agent.get_longs(pool=pool):
        if long.maturity_time <= chain.block_time():
            print(f"Close long-{long.maturity_time} on {pool.name}")
            agent.close_long(maturity_time=long.maturity_time, bonds=long.balance, pool=pool)
    # Close all mature shorts
    for short in agent.get_shorts(pool=pool):
        if short.maturity_time <= chain.block_time():
            print(f"Close short-{short.maturity_time} on {pool.name}")
            agent.close_short(maturity_time=short.maturity_time, bonds=short.balance, pool=pool)

## Write policies in Python


In [ ]:
from agent0 import HyperdriveBasePolicy, open_long_trade
from fixedpointmath import FixedPoint
from dataclasses import dataclass

In [ ]:
class OpenLongPolicy(HyperdriveBasePolicy):
    @dataclass(kw_only=True)
    class Config(HyperdriveBasePolicy.Config):
        fixed_rate_threshold: FixedPoint
        open_long_amount: FixedPoint

    def action(self, interface, wallet):
        """Get agent actions for the current block

        Action fn returns the trades to be executed
        at a given moment in time.
        """
        done_trading = False  # Never done trading

        # If no longs in wallet, we check our fixed rate
        # threshold and open the long if threshold reached.
        if len(wallet.longs) == 0:
            if interface.calc_spot_rate() > self.config.fixed_rate_threshold:
                return [open_long_trade(self.config.open_long_amount)], done_trading

        # We don't do any trades otherwise
        return [], done_trading

In [ ]:
agent.set_active(
    policy=OpenLongPolicy,
    policy_config=OpenLongPolicy.Config(
        fixed_rate_threshold=FixedPoint(0.06),
        open_long_amount=FixedPoint("100_000"),
    ),
)
agent.execute_policy_action(pool=registered_pools[0])